# **INTRODUCTION**

* * *

This workflow consists of several processes used for creating the so called 'berggrafiek' on the dashboard. 

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Views](#views)**
3. **[Stored Procedures](#stored-procedures)**
4. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/dataflows/vws_vaccine_deliveries_administered.ipynb"
    ]
}
```

# **TABLES**

---

### STAGING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE [object_id] = OBJECT_ID('VWSSTAGE.MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'))
CREATE TABLE [VWSSTAGE].[MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [CATEGORY] [VARCHAR](255) NULL,
    [KEY] [VARCHAR](255) NULL,
    [VALUE] [VARCHAR](255) NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (GETDATE()) NOT NULL
);
GO

-- 3) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_VWS_VACCINE_SHOTS_PLANNED]
    ON [VWSSTAGE].[MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_VWS_VACCINE_SHOTS_PLANNED]
    ON [VWSSTAGE].[MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED](
        [DATE_LAST_INSERTED] DESC
    );
GO

# **VIEWS**

---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED]
AS
	WITH CTE
	AS (
		SELECT
			[CATEGORY],
			[KEY],
			[VALUE],
			[DATE_LAST_INSERTED]
		FROM [VWSSTAGE].[MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED] WITH(NOLOCK)
		WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED] WITH(NOLOCK))
			AND UPPER([CATEGORY]) = 'PRIKKEN_GEPLAND'
	)
	SELECT TOP(1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'SHOTS_PLANNED_TOTAL')  AS [DOSES],
		[dbo].[CONVERT_DATETIME_TO_UNIX](CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_START_UNIX'), 105)) AS [DATE_START_UNIX],
		[dbo].[CONVERT_DATETIME_TO_UNIX](CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_END_UNIX'), 105)) AS [DATE_END_UNIX],
		[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
	FROM CTE
GO

# **DROP ARTIFACTS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED',
        @proto_name VARCHAR(50) = 'NL';

-- 1) DROP PROTOS CONFIGURATION(S).....
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [VIEW_ID] IN (
    SELECT [ID] AS VID FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_VWS_VACCINE_ADMINISTERED_PLANNED_TEMP',
        'VWSDEST.V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'
    ))
    AND [PROTO_ID] IN (
        SELECT [ID] FROM [DATATINO_PROTO_1].[PROTOS] protos 
        WHERE [NAME] = @proto_name
    ) AND [NAME] IN (
        'vaccine_administered_planned'
    );

DELETE FROM [DATATINO_PROTO_1].[VIEWS]
WHERE [ID] IN (
    SELECT 
        [ID] AS VID 
    FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_VWS_VACCINE_ADMINISTERED_PLANNED_TEMP',
        'VWSDEST.V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'
    )
)

-- 2) DROP PROCESS(ES)......
;WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        CONCAT('PROCESS_', @workflow_name)
    ) AND v_workflows.[NAME] = @workflow_name
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);

-- 4) DROP SOURCE(S).....
DELETE FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] IN (
    CONCAT('SOURCE_', @workflow_name)
);

-- 5) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );

-- 6) DROP DATAFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[H_DATAFLOWS]
WHERE [NAME] IN (
    CONCAT('PROCESS_', @workflow_name),
    @workflow_name
);

DELETE  [DATATINO_ORCHESTRATOR_1].[DATAFLOWS]
WHERE [NAME] IN (
    CONCAT('PROCESS_', @workflow_name),
    @workflow_name
)
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 12 * * MON-FRI', -- AT 02:00 PM, FROM MONDAY TO FRIDAY
    @active = @is_active;
GO

### SOURCES

> STAGING SOURCES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET WORKFLOW NAME
DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50),
        @delimiter_type VARCHAR(50),
        @source_type VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/VWS/Handmatige_invoer_verwachte_prikken.csv'
    WHEN 'acceptance' THEN 'datafiles/VWS/Handmatige_invoer_verwachte_prikken.csv'
    ELSE 'datafiles/VWS/Handmatige_invoer_verwachte_prikken.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'N/A'
    WHEN 'acceptance' THEN 'N/A'
    ELSE 'N/A'
END;

SET @delimiter_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'SemiColon'
    WHEN 'acceptance' THEN 'SemiColon'
    ELSE 'SemiColon'
END;

SET @source_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'CsvFile'
    WHEN 'acceptance' THEN 'CsvFile'
    ELSE 'CsvFile'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,    
    @source_columns = 'CATEGORY|KEY|VALUE',
    @target_columns = 'CATEGORY|KEY|VALUE',
    @target_name = @target_name,
    @source_type = @source_type,
    @location_type = @location_type,
    @delimiter_type = @delimiter_type,
    @security_profile= @security_profile;
GO

### PROCESSES

> STAGING SOURCES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET WORKFLOW NAME
DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description =  CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;
GO

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- -- 1) SET WORKFLOW NAME
-- DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'

-- -- 2) SET ENVIRONMENTAL VARIABLES.....
-- DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED',
--         @view_description VARCHAR(256),
--         @item_name VARCHAR(256) = 'vaccine_administered_planned',
--         @config_description VARCHAR(256),
--         @constraint_value VARCHAR(50) = NULL,
--         @constraint_key_name VARCHAR(50) = NULL,
--         @grouped_key_name VARCHAR(50) = null,
--         @grouped_last_update_name VARCHAR(50) = null,
--         @proto_name VARCHAR(50) = 'NL',
--         @columns VARCHAR(256) = '*',
--         @layout_type_id INT = 1, -- LASTVALUES
--         @last_update_name VARCHAR(50) = 'DATE_END_UNIX',
--         @is_active INT;
        
-- SET @is_active = CASE LOWER('#{ Environment }#')
--     WHEN 'production' THEN 1
--     WHEN 'acceptance' THEN 1
--     ELSE 1
-- END;

-- SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
-- SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- -- 2) DETERMINE VIEW ID & CONFIGURATION ID
-- DECLARE @constrained INT,
--         @grouped INT,
--         @view_id BIGINT,
--         @config_id BIGINT;

-- SET @constrained = CASE 
--     WHEN @constraint_key_name IS NULL THEN 0
--     ELSE 1
-- END;
-- SET @grouped = CASE 
--     WHEN @grouped_key_name IS NULL THEN 0
--     ELSE 1
-- END;

-- SELECT 
--     @view_id = [ID]
-- FROM [DATATINO_PROTO_1].[VIEWS]
-- WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
--     AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
--     AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
--     AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
--     AND [NAME] = @view_name;
    
-- SELECT
--     @config_id = configs.[ID]
-- FROM [DATATINO_PROTO_1].[VIEWS] views
-- INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
--     AND configs.[NAME] = @item_name
--     AND configs.[VIEW_ID] = @view_id
-- INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
--     AND protos.[NAME] = @proto_name
-- WHERE views.[NAME] = @view_name;

-- -- 3) UPSERT PROTO VIEW(S).....
-- EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
--     @id = @view_id,
--     @view_name = @view_name,
--     @description = @view_description,
--     @last_update_name = @last_update_name,
--     @constraint_key_name = @constraint_key_name,
--     @constraint_value = @constraint_value,
--     @grouped_key_name = @grouped_key_name,
--     @grouped_last_update_name = @grouped_last_update_name;

-- -- 4) UPSERT PROTO CONFIGURATION(S).....
-- EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
--     @id = @config_id,
--     @proto_name = @proto_name,
--     @description =  @config_description,
--     @view_name = @view_name,
--     @item_name = @item_name,
--     @constrained = @constrained,
--     @grouped = @grouped,
--     @columns = @columns,
--     @mapping = '=LOWER()',
--     @layout_type_id = @layout_type_id,
--     @active = @is_active,
--     @constraint_key_name = @constraint_key_name,
--     @constraint_value = @constraint_value,
--     @grouped_key_name = @grouped_key_name,
--     @grouped_last_update_name = @grouped_last_update_name;
-- GO